<a href="https://colab.research.google.com/github/YanzhaoZ/PreBit/blob/main/Price_Base_Model_New_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install yfinance
import yfinance as yf
import pandas as pd


In [83]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler


# Loading Price Data

In [3]:
start_date ='2015-01-01'
end_date = '2019-12-31'
price = yf.download("BTC-USD", start=start_date, end=end_date)


[*********************100%***********************]  1 of 1 completed


In [4]:
def get_technical_indicators(price):
    # Create 7 and 21 days Moving Average
    dataset = price.copy()

    dataset['ma7'] = dataset['Close'].rolling(window=7).mean()
    dataset['ma21'] = dataset['Close'].rolling(window=21).mean()
    
    # Create MACD
    #dataset['26ema'] = pd.ewma(dataset['Close'], span=26)
    dataset['26ema'] = dataset['Close'].ewm(span=26).mean()
    #dataset['12ema'] = pd.ewma(dataset['Close'], span=12)
    dataset['12ema'] = dataset['Close'].ewm(span=12).mean()
    dataset['MACD'] = (dataset['12ema']-dataset['26ema'])

    # Create Bollinger Bands
    #dataset['20sd'] = pd.stats.moments.rolling_std(dataset['Close'],20)
    dataset['20sd'] = dataset["Close"].rolling(window=20).std()
    dataset['upper_band'] = dataset['ma21'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma21'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = dataset['Close'].ewm(com=0.5).mean()
    
    # Create Momentum
    #dataset['momentum'] = dataset['Close']-1

    # Create high-low spred
    dataset['spread'] = dataset['High'] - dataset['Low']
    
    return dataset

In [5]:
 # to be finished 
def get_related_asset (price, start_date, end_date):
    other_price = yf.download("ETH-USD", start=start_date, end=end_date)

    price['eth']=other_price["Close"]
    #there are nan values because the price datarange is not the same. so I should probalby fill the value by non-0, coz i need to do division later...
    #let's fill it with the next valid value 
    price.eth.fillna(method='bfill', inplace= True, axis=0)

    return price



In [6]:
def get_label (price,threshold):


    price['change']=price.shift(-1).High/price.Close -1 
    price['change_label']=price['change'].apply (lambda x: x> threshold)

    #convert True/False to 1/0
    class2idx = {True: 1, False:0}
    price['change_label'].replace(class2idx, inplace=True)

    return price

In [7]:
def normalise_close(price):
    
    df = price.copy()
    for key in df.keys():
        if not key in ['change','change_label','Volume','MACD','20sd','spread','eth']:
            df[key]=df[key]/price['Close'].shift(1) - 1
        
    df['Volume']=df['Volume']/price['Volume'].shift(1)-1
    df['eth']  = df['eth']/price['eth'].shift(1)-1
    df['MACD'] = df['MACD']/price['Close'].shift(1)
    df['20sd'] = df['20sd']/price['Close'].shift(1)
    df['spread'] = df['spread']/price['Close'].shift(1)



    return df


In [8]:
def get_ma_feature (price,threshold):
    price['ma_feature']=price['ma7'].apply(lambda x: x > threshold)
    
    class2idx = {True: 1, False:0}
    price['ma_feature'].replace(class2idx, inplace=True)
    return price

In [9]:
#Here I want to define a function to return a new dataframe, that's INDEPENDANT, and produce relative price. 
def process_price (original_df,threshold, start_date,end_date):
    #get the indicators
    df = get_technical_indicators(original_df)

    #get related asset
    df = get_related_asset(df,start_date,end_date)

    #get label
    df = get_label(df, threshold)

    #normalise the data
    df_normalised_close = normalise_close(df)

    #get ma label
    df_normalised_close = get_ma_feature(df_normalised_close, threshold)

    #return both normalized, and un-normalized data (w/o ma label). 
    return df, df_normalised_close





In [10]:
test_df, test_df_norm = process_price(price,0.05,start_date,end_date)

[*********************100%***********************]  1 of 1 completed


In [11]:
test_df.head()

,Open,High,Low,Close,Adj Close,Volume,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,ema,spread,eth,change,change_label
Date,,,,,,,,,,,,,,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8036550,NaN,NaN,314.248993,314.248993,0.000000,NaN,NaN,NaN,314.248993,6.432007,2.77212,0.005060,0
2015-01-02,314.079010,315.838989,313.565002,315.032013,315.032013,7860650,NaN,NaN,314.655561,314.673129,0.017568,NaN,NaN,NaN,314.836258,2.273987,2.77212,0.000375,0
2015-01-03,314.846008,315.149994,281.082001,281.082001,281.082001,33054400,NaN,NaN,302.592907,301.562504,-1.030403,NaN,NaN,NaN,291.467926,34.067993,2.77212,0.021873,0
2015-01-04,281.145996,287.230011,257.612000,264.195007,264.195007,55629100,NaN,NaN,291.858532,289.767045,-2.091487,NaN,NaN,NaN,273.058706,29.618011,2.77212,0.053544,1
2015-01-05,265.084015,278.341003,265.084015,274.473999,274.473999,43962800,NaN,NaN,287.826987,285.611979,-2.215008,NaN,NaN,NaN,274.006134,13.256989,2.77212,0.047651,0


In [12]:
test_df_norm.head()

,Open,High,Low,Close,Adj Close,Volume,ma7,ma21,26ema,12ema,MACD,20sd,upper_band,lower_band,ema,spread,eth,change,change_label,ma_feature
Date,,,,,,,,,,,,,,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005060,0,0
2015-01-02,-0.000541,0.005060,-0.002177,0.002492,0.002492,-0.021888,NaN,NaN,0.001294,0.001350,0.000056,NaN,NaN,NaN,0.001869,0.007236,0.0,0.000375,0,0
2015-01-03,-0.000590,0.000375,-0.107767,-0.107767,-0.107767,3.205047,NaN,NaN,-0.039485,-0.042756,-0.003271,NaN,NaN,NaN,-0.074799,0.108141,0.0,0.021873,0,0
2015-01-04,0.000228,0.021873,-0.083499,-0.060079,-0.060079,0.682956,NaN,NaN,0.038339,0.030899,-0.007441,NaN,NaN,NaN,-0.028544,0.105371,0.0,0.053544,1,0
2015-01-05,0.003365,0.053544,0.003365,0.038907,0.038907,-0.209716,NaN,NaN,0.089449,0.081065,-0.008384,NaN,NaN,NaN,0.037136,0.050179,0.0,0.047651,0,0


# Some analysis with MA only

In [ ]:
(test_df_norm['change_label']==test_df_norm['ma_feature']).value_counts()

True     1537
False     289
dtype: int64

In [ ]:
test_df_norm['change_label'].value_counts()

0    1608
1     218
Name: change_label, dtype: int64

In [ ]:
#why not get a confusion matrix with these two labels and check it out. 
accuracy = (test_df_norm['change_label']==test_df_norm['ma_feature']).sum() / len(test_df_norm['change_label']==test_df_norm['ma_feature'])
print ('accuracy:', accuracy)
print (confusion_matrix(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))
print (classification_report(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))

#The true label, accuracy is only 0.27, and recall rate is quite low to be honest. 

accuracy: 0.8417305585980285
[[1495  113]
 [ 176   42]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      1608
           1       0.27      0.19      0.23       218

    accuracy                           0.84      1826
   macro avg       0.58      0.56      0.57      1826
weighted avg       0.82      0.84      0.83      1826



In [ ]:
#for 21day ma features
#why not get a confusion matrix with these two labels and check it out. 
accuracy = (test_df_norm['change_label']==test_df_norm['ma_feature']).sum() / len(test_df_norm['change_label']==test_df_norm['ma_feature'])
print ('accuracy:', accuracy)
print (confusion_matrix(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))
print (classification_report(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))

#The true label, accuracy is only 0.27, and recall rate is quite low to be honest. 

accuracy: 0.7431544359255202
[[1301  307]
 [ 162   56]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.85      1608
           1       0.15      0.26      0.19       218

    accuracy                           0.74      1826
   macro avg       0.52      0.53      0.52      1826
weighted avg       0.80      0.74      0.77      1826



In [ ]:
#for ema features
#why not get a confusion matrix with these two labels and check it out. 
accuracy = (test_df_norm['change_label']==test_df_norm['ma_feature']).sum() / len(test_df_norm['change_label']==test_df_norm['ma_feature'])
print ('accuracy:', accuracy)
print (confusion_matrix(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))
print (classification_report(test_df_norm['change_label'].values, test_df_norm['ma_feature'].values))

#The true label, accuracy is only 0.27, and recall rate is quite low to be honest. 

accuracy: 0.864184008762322
[[1561   47]
 [ 201   17]]
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1608
           1       0.27      0.08      0.12       218

    accuracy                           0.86      1826
   macro avg       0.58      0.52      0.52      1826
weighted avg       0.81      0.86      0.83      1826



In [ ]:
#so bascially 7day ma works best among these 3. got it. 


# Format input for Torch Models

Creating two dataset classes for future use, 1 is for classification tast, the other for Regression task

In [34]:
class ClassifierDataset(Dataset):
    def __init__(self,df):
        self.df = df
        self.X, self.Y = self.clean_df()


    def __len__(self):
        return len(self.Y)


    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


    def clean_df(self):
        #drop NaN values, and also load the values into torch.tensor. 
        df = self.df.dropna()
        y_values = df.change_label.values
        df = df.drop (['change','change_label'],axis=1)
        x_values = df.values

        return torch.from_numpy(x_values).float(), torch.from_numpy(y_values).long()


      


In [33]:
class RegressionDataset(Dataset):
    def __init__(self,df):
        self.df = df
        self.X, self.Y = self.clean_df()


    def __len__(self):
        return len(self.Y)


    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


    def clean_df(self):
        #drop NaN values, and also load the values into torch.tensor.
        df = self.df.dropna()
        y_values = df.change.values
        df = df.drop (['change','change_label'],axis=1)
        x_values = df.values

        return torch.from_numpy(x_values).float(), torch.from_numpy(y_values).float()



In [75]:
# define a function for train_test split, and initiate dataset? 
def split_train_test (df, threshold):
    length = len(df)
    split_number  = round(threshold*length)
    return df[0:split_number], df[split_number:]

In [79]:
#for classification dataset construction
test_df_train,test_df_eval = split_train_test(test_df_norm, 0.7)
train_dataset = ClassifierDataset(test_df_train)
eval_dataset = ClassifierDataset (test_df_eval)

In [ ]:
#for Regression dataset construction

# test_df_train,test_df_eval = split_train_test(test_df_norm, 0.7)
# train_dataset = RegressionDataset(test_df_train)
# eval_dataset = RegressionDataset (test_df_eval)

In [81]:
eval_dataset[0]

(tensor([-0.0009,  0.0467, -0.0125,  0.0358,  0.0358, -0.0817, -0.0207,  0.0035,
          0.0373, -0.0022, -0.0395,  0.0413,  0.0861, -0.0791,  0.0226,  0.0592,
          0.0472,  0.0000]), tensor(0))

In [44]:
#Now let's try to flow the data into dataloader 
train_loader = DataLoader(train_dataset, batch_size = 64, )
eval_loader = DataLoader(eval_dataset, batch_size = 64, )
# add in shuffle/sampler options later

In [45]:
#next(iter(train_loader))

[tensor([[ 2.9815e-04,  7.7955e-02, -4.8741e-04,  ...,  7.8442e-02,
           0.0000e+00,  0.0000e+00],
         [ 1.8731e-03,  4.4611e-02, -2.0406e-03,  ...,  4.6651e-02,
           0.0000e+00,  0.0000e+00],
         [ 4.7553e-04,  6.1652e-03, -3.5175e-02,  ...,  4.1340e-02,
           0.0000e+00,  0.0000e+00],
         ...,
         [-2.4258e-04,  3.4845e-02, -2.3194e-02,  ...,  5.8039e-02,
           0.0000e+00,  0.0000e+00],
         [-6.1106e-04,  9.8598e-04, -8.4670e-02,  ...,  8.5656e-02,
           0.0000e+00,  0.0000e+00],
         [ 7.6427e-03,  1.4638e-02, -3.6971e-02,  ...,  5.1609e-02,
           0.0000e+00,  1.0000e+00]]),
 tensor([ 0.0446,  0.0062,  0.0658,  0.0292,  0.2194,  0.0073,  0.0116,  0.0205,
          0.0400,  0.0313,  0.0649,  0.0670,  0.0324,  0.0123,  0.0553,  0.0617,
          0.0361,  0.0074,  0.0025,  0.0077,  0.0162,  0.0138,  0.0834,  0.1036,
          0.0322,  0.0200,  0.0510,  0.0026,  0.0269,  0.0284,  0.0473,  0.0076,
          0.0175,  0.0042,  0.

In [ ]:
# for i, batch in enumerate(train_loader):
#     print (i)

In [ ]:
#weighed random sampler test. 

import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, TensorDataset
train_dataset = torch.tensor([1, 1 , 1, 1, 1, 1, 1, 1, 1, 0])


class_weights_all = [0.1, 0.1, 0.1, 0.1,0.1, 0.1, 0.1, 0.1,0.1, 0.9]

weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=10,
    replacement=True #if True, sampler will draw repeating inputs, False will have 0 repeats. 
)

BATCH_SIZE = 5
dataset = TensorDataset(train_dataset)
train_loader = DataLoader(dataset,
                          batch_size=BATCH_SIZE,
                          sampler=weighted_sampler
)
for batch in train_loader:
    print(batch)



# Building Torch Model(s)

In [85]:
#let's first try a simple FC NN
class FFNN (nn.Module):
    def __init__(self, input_size, num_classes,num_hidden, hidden_dim ):
        super().__init__()

        assert num_layers > 0 

        self.fc1 = nn.Linear(input_size,hidden_dim)
        self.fc2 = nn.Linear(hidden_dim,hidden_dim)

        self.hidden_layers = nn.ModuleList([])
        self.hidden_layers.append (self.fc1)
        for i in range (num_hidden -1 ):
            self.hidden_layers.append(self.fc2)
        
        self.final_layer = nn.Linear (hidden_dim,num_classes)
        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()

    def forward(self,x):
        for hidden_layer in self.hidden_layers:
            x = hidden_layer(x)
            x = self.dropout(x)
            x = self.relu(x)
        
        out = self.final_layer(x)
        out_dist = F.log_softmax(out, dim= -1) #why is it -1, ok, coz it's batch x input x class. 
        
        return out_dist



In [ ]:
#try initiating the model
INPUT_SIZE = 
NUM_CLASSES = 
NUM_HIDDEN = 
HIDDEN_DIM = 

model = FFNN(INPUT_SIZE, NUM_CLASSES, NUM_HIDDEN, HIDDEN_DIM)
